# pytorch 101

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


# Creat Model

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28,512),nn.ReLU(),nn.Linear(512,512),nn.ReLU(),nn.Linear(512,10),nn.ReLU())
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)

print(model)



Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# Optimizing the Model Parameters


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y  = X.to(device),y.to(device)
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 ==0 :
            loss,current = loss.item(),batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss  += loss_fn(pred,y).item()
            correct += (pred.argmax(1)== y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.581136  [    0/60000]
loss: 1.578499  [ 6400/60000]
loss: 1.388895  [12800/60000]
loss: 1.443964  [19200/60000]
loss: 1.454681  [25600/60000]
loss: 1.535340  [32000/60000]
loss: 1.357982  [38400/60000]
loss: 1.398639  [44800/60000]
loss: 1.518858  [51200/60000]
loss: 1.410977  [57600/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.419901 

Epoch 2
-------------------------------
loss: 1.476831  [    0/60000]
loss: 1.487324  [ 6400/60000]
loss: 1.282590  [12800/60000]
loss: 1.344944  [19200/60000]
loss: 1.355325  [25600/60000]
loss: 1.455056  [32000/60000]
loss: 1.269731  [38400/60000]
loss: 1.323484  [44800/60000]
loss: 1.440022  [51200/60000]
loss: 1.342420  [57600/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.345454 

Epoch 3
-------------------------------
loss: 1.398685  [    0/60000]
loss: 1.421350  [ 6400/60000]
loss: 1.204431  [12800/60000]
loss: 1.271617  [19200/60000]
loss: 1.285724  [25600/60000]
loss: 1.394904  [32000/600

# Saving Model

In [20]:
torch.save(model.state_dict(),"./model/101.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)